In [20]:
%serialconnect --port /dev/tty.usbmodem14601 --baud=115200

serial exception on close write failed: [Errno 6] Device not configured
Connecting to --port=/dev/tty.usbmodem14601 --baud=115200 
Ready.


In [21]:
import board
import busio
import digitalio

import adafruit_rfm9x


# Define radio parameters.
RADIO_FREQ_MHZ = 915.0  # Frequency of the radio in Mhz. Must match your
                        # module! Can be a value like 915.0, 433.0, etc.

# Define pins connected to the chip, use these if wiring up the breakout according to the guide:
#CS = digitalio.DigitalInOut(board.D5)
#RESET = digitalio.DigitalInOut(board.D6)
# Or uncomment and instead use these if using a Feather M0 RFM9x board and the appropriate
# CircuitPython build:
CS = digitalio.DigitalInOut(board.RFM9X_CS)
RESET = digitalio.DigitalInOut(board.RFM9X_RST)

# Define the onboard LED
LED = digitalio.DigitalInOut(board.D13)
LED.direction = digitalio.Direction.OUTPUT

# Initialize SPI bus.
spi = busio.SPI(board.SCK, MOSI=board.MOSI, MISO=board.MISO)

In [22]:
print(dir(board))
print(board.RFM9X_CS)

['A0', 'A1', 'A2', 'A3', 'A4', 'A5', 'BATTERY', 'D0', 'D1', 'D10', 'D11', 'D12', 'D13', 'D5', 'D6', 'D9', 'I2C', 'MISO', 'MOSI', 'RFM9X_CS', 'RFM9X_D0', 'RFM9X_RST', 'RX', 'SCK', 'SCL', 'SDA', 'SPI', 'TX', 'UART', 'VOLTAGE_MONITOR']
microcontroller.pin.RFM9X_CS


In [24]:
print(dir(adafruit_rfm9x))
# Initialze RFM radio
rfm9x = adafruit_rfm9x.RFM9x(spi, CS, RESET, RADIO_FREQ_MHZ)

# Note that the radio is configured in LoRa mode so you can't control sync
# word, encryption, frequency deviation, or other settings!

# You can however adjust the transmit power (in dB).  The default is 13 dB but
# high power radios like the RFM95 can go up to 23 dB:
rfm9x.tx_power = 23

# Send a packet.  Note you can only send a packet up to 252 bytes in length.
# This is a limitation of the radio packet size, so if you need to send larger
# amounts of data you will need to break it into smaller send calls.  Each send
# call will wait for the previous one to finish before continuing.
rfm9x.send(bytes("Hello world!\r\n","utf-8"))
print('Sent Hello World message!')

# Wait to receive packets.  Note that this library can't receive data at a fast
# rate, in fact it can only receive and process one 252 byte packet at a time.
# This means you should only use this for low bandwidth scenarios, like sending
# and receiving a single message at a time.
print('Waiting for packets...')

while True:
    packet = rfm9x.receive()
    # Optionally change the receive timeout from its default of 0.5 seconds:
    #packet = rfm9x.receive(timeout=5.0)
    # If no packet was received during the timeout then None is returned.
    if packet is None:
        # Packet has not been received
        LED.value = False
        print('Received nothing! Listening again...')
    else:
        # Received a packet!
        LED.value = True
        # Print out the raw bytes of the packet:
        print('Received (raw bytes): {0}'.format(packet))
        # And decode to ASCII text and print it too.  Note that you always
        # receive raw bytes and need to convert to a text format like ASCII
        # if you intend to do string processing on your data.  Make sure the
        # sending side is sending ASCII data before you try to decode!
        packet_text = str(packet, 'ascii')
        print('Received (ASCII): {0}'.format(packet_text))
        # Also read the RSSI (signal strength) of the last received message and
        # print it.
        rssi = rfm9x.rssi
        print('Received signal strength: {0} dB'.format(rssi))

['__file__', '__name__', 'const', 'digitalio', 'time', 'warn', 'spidev', '__version__', '__repo__', '_RH_RF95_FXOSC', '_RH_RF95_FSTEP', 'SLEEP_MODE', 'STANDBY_MODE', 'FS_TX_MODE', 'TX_MODE', 'FS_RX_MODE', 'RX_MODE', 'RFM9x']


Traceback (most recent call last):
  File "<stdin>", line 3, in <module>
  File "adafruit_rfm9x.py", line 374, in __init__
  File "adafruit_rfm9x.py", line 372, in __init__
RuntimeError: Failed to configure radio for LoRa mode, check wiring!
